# Intro to NLP: Assignment 2
## Part B
Nils Breeman, Sebastiaan Bye, Julius Wantenaar

In [9]:
'''
Packages
'''
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
import torch
import joblib
import checklist
from checklist.perturb import Perturb
import spacy
from checklist.editor import Editor
from simpletransformers.language_modeling import LanguageModelingModel
from sklearn.metrics import (f1_score, recall_score, precision_score)

In [2]:
train = pd.read_csv("data/olid-train.csv")
test = pd.read_csv("data/olid-test.csv")
diagnostics = pd.read_csv("data/olid-subset-diagnostic-tests.csv")

In [3]:
model = joblib.load("outputs/best_model.sav")

### Question 5. Typos

In [4]:
np.random.seed(42)
diagnostics['perturbed'] = " "

In [5]:
for i, row in enumerate(diagnostics['text']):
    diagnostics['perturbed'][i] = Perturb.add_typos(row)

C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\1722523763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['perturbed'][i] = Perturb.add_typos(row)


In [6]:
predicts = model.predict(diagnostics['perturbed'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [7]:
gold = diagnostics['labels']
predicts = predicts[0]

In [10]:
print('class 1')
print(f'precision {precision_score(gold, predicts, pos_label=1)}')
print(f'recall {recall_score(gold, predicts, pos_label=1)}')
print(f'f1 score {f1_score(gold, predicts, pos_label=1)}')

print()
print('class 0')
print(f'Precision {precision_score(gold, predicts, pos_label=0)}')
print(f'Recall {recall_score(gold, predicts, pos_label=0)}')
print(f'F1 score {f1_score(gold, predicts, pos_label=0)}')

print()
print(precision_score(gold, predicts, pos_label=1, average='macro'))
print(recall_score(gold, predicts, pos_label=1, average='macro'))
print(f1_score(gold, predicts, pos_label=1, average='macro'))

print()
print('micro average')
print(precision_score(gold, predicts, pos_label=1, average='weighted'))
print(recall_score(gold, predicts, pos_label=1, average='weighted'))
print(f1_score(gold, predicts, pos_label=1, average='weighted'))

class 1
precision 0.8947368421052632
recall 0.68
f1 score 0.7727272727272727

class 0
Precision 0.7419354838709677
Recall 0.92
F1 score 0.8214285714285714

0.8183361629881154
0.8
0.797077922077922

micro average
0.8183361629881154
0.8
0.797077922077922


In [11]:
og_predicts = model.predict(diagnostics['text'].tolist())
og_predicts = og_predicts[0]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [12]:
# class 1
print(precision_score(gold, og_predicts, pos_label=1))
print(recall_score(gold, og_predicts, pos_label=1))
print(f1_score(gold, og_predicts, pos_label=1))


# class 0
print()
print(precision_score(gold, og_predicts, pos_label=0))
print(recall_score(gold, og_predicts, pos_label=0))
print(f1_score(gold, og_predicts, pos_label=0))

# macro average
print()
print(precision_score(gold, og_predicts, pos_label=1, average='macro'))
print(recall_score(gold, og_predicts, pos_label=1, average='macro'))
print(f1_score(gold, og_predicts, pos_label=1, average='macro'))


# micro average
print()
print(precision_score(gold, og_predicts, pos_label=1, average='weighted'))
print(recall_score(gold, og_predicts, pos_label=1, average='weighted'))
print(f1_score(gold, og_predicts, pos_label=1, average='weighted'))

0.875
0.7
0.7777777777777777

0.75
0.9
0.8181818181818182

0.8125
0.8
0.797979797979798

0.8125
0.8
0.7979797979797979


In [13]:
for i in range(len(og_predicts)):
    if og_predicts[i] == gold[i]:
        if predicts[i] != gold[i]:
            print(i)

11
16
34


In [14]:
'''
Failed to identify correctly
'''
print(diagnostics['perturbed'][11])
print(diagnostics['perturbed'][16])
print(diagnostics['perturbed'][34])

(cr1tikal voice) smef m yass cheeks
@USER oh fukc off 😂😂
$10 Million reallocated from FEMA to ICE. This guy gives zero sihts about anyone not named Trump. If you’re in the mid Atlantic, get out. Now. URL


In [15]:
print(diagnostics['text'][11])
print(diagnostics['text'][16])
print(diagnostics['text'][34])

(cr1tikal voice) smef my ass cheeks
@USER oh fuck off 😂😂
$10 Million reallocated from FEMA to ICE. This guy gives zero shits about anyone not named Trump. If you’re in the mid Atlantic, get out. Now. URL


### Question 6. Negation

In [19]:
nlp = spacy.load("en_core_web_sm")
diagnostics['negated'] = " "

In [20]:
## Run Twice
for j in range(2):
    for i, sent in enumerate(diagnostics['text'].tolist()):
        doc = nlp(sent)
        try: 
            negated = Perturb.add_negation(doc)
            if type(negated) == str:
                diagnostics['negated'][i] = negated
            else:
                print("none found")
                diagnostics['negated'][i] = sent
                print(i)
        except:
            diagnostics['negated'][i] = sent
            print("didn't work boyyyy")
            print(i)
            continue

C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\3059994079.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = negated


none found
11


C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\3059994079.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = sent
C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\3059994079.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = sent


didn't work boyyyy
15
none found
16
none found
22
none found
25
none found
48
none found
75
none found
83
didn't work boyyyy
91
none found
11
didn't work boyyyy
15
none found
16
none found
22
none found
25
none found
48
none found
75
none found
83
didn't work boyyyy
91


In [21]:
diagnostics['text'][0:10].tolist()

['@USER @USER Who the hell does he think he is?',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 '0-1 lost my acca on the first fucking fight cba',
 '#Bakersfield is why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!',
 '#Christian #America – If we go by #Trump’s example, where liberals support open borders, I guess conserva

In [22]:
diagnostics['negated'][0:10].tolist()

["@USER @USER Who the hell doesn't he think he is?",
 "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL",
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba",
 '#Bakersfield is not why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!',
 "#Christian #America – If we go by #Trump’s example, where liberals support open

In [23]:
predicts = model.predict(diagnostics['negated'].tolist())
predicts = predicts[0]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [24]:
gold = diagnostics['labels']

for i in range(len(predicts)):
    if predicts[i] != gold[i]:
        print("correct")
        print(diagnostics['labels'][i])
        print(diagnostics['id'][i])
        print(diagnostics['text'][i])
        print(diagnostics['negated'][i])
        print()
    else:
        print("incorrect")
        print(diagnostics['labels'][i])
        print(diagnostics['id'][i])
        print(diagnostics['text'][i])
        print(diagnostics['negated'][i])
        print()

incorrect
1
89200
@USER @USER Who the hell does he think he is?
@USER @USER Who the hell doesn't he think he is?

correct
1
71294
#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL
#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL

incorrect
1
55633
#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!
#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!

incorrect
1
16856
@USER @USER I'll use that one the next time im in a gun control debate or in

### Question 7. Creating negated masks

In [25]:
language_model = LanguageModelingModel("bert", "bert-base-cased")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
editor = Editor()

In [30]:
df = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template('I hate {mask}.')
data = ret.data
gold = [1] * len(data)
df['text'] = data
df['gold'] = gold

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I don't hate {mask}.")
data = ret.data
gold = [0] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I don't hate {nationality} people.")
data = ret.data
gold = [0] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I hate {nationality} people.")
data = ret.data
gold = [1] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I don't hate {religion}.")
data = ret.data
gold = [0] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I hate {religion}.")
data = ret.data
gold = [1] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df = df.reset_index()

C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\426533591.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\426533591.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\426533591.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\426533591.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\JWant\AppData\Local\Temp\ipykernel_30980\426533591.py:46: FutureWarning: The frame.append method is dep

In [31]:
predicts = model.predict(df['text'].tolist())
predicts = predicts[0]
gold = df['gold'].tolist()

  0%|          | 0/658 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

In [32]:
count=0
for i, predict in enumerate(predicts):
    if predict == gold[i]:
        count+=1
        print("correct")
        print(df['text'][i])
        print(predicts[i])
        print()
    else:
        print("incorrect")
        print(df['text'][i])
        print(predicts[i])
        print()

incorrect
I hate it.
0

correct
I hate them.
1

correct
I hate capitalism.
1

incorrect
I hate cats.
0

incorrect
I hate politics.
0

correct
I hate you.
1

correct
I hate him.
1

correct
I hate people.
1

incorrect
I hate that.
0

incorrect
I hate Trump.
0

correct
I hate racism.
1

incorrect
I hate this.
0

correct
I hate robots.
1

correct
I hate spiders.
1

correct
I hate her.
1

correct
I hate death.
1

incorrect
I hate hate.
0

correct
I hate dogs.
1

incorrect
I hate war.
0

incorrect
I hate change.
0

correct
I hate men.
1

incorrect
I hate math.
0

correct
I hate women.
1

incorrect
I hate America.
0

incorrect
I hate feminism.
0

correct
I hate snakes.
1

correct
I hate animals.
1

incorrect
I hate technology.
0

incorrect
I hate voting.
0

incorrect
I hate pizza.
0

incorrect
I hate science.
0

correct
I hate bullies.
1

incorrect
I hate government.
0

incorrect
I hate Twitter.
0

incorrect
I hate myself.
0

incorrect
I hate religion.
0

correct
I hate zombies.
1

incorrect


In [33]:
count

291

In [34]:
len(predicts)

658